Import

In [ ]:
import sagemaker
import boto3
import sys
import os
import glob
import re
import subprocess
import numpy as np
from IPython.display import HTML
import time
from time import gmtime, strftime
sys.path.append("common")
from misc import get_execution_role, wait_for_s3_object
from docker_utils import build_and_push_docker_image
from sagemaker.rl import RLEstimator, RLToolkit, RLFramework

Setup S3 bucket

In [ ]:
sage_session = sagemaker.session.Session()
s3_bucket = sage_session.default_bucket()  
s3_output_path = 's3://{}/'.format(s3_bucket)
print("S3 bucket path: {}".format(s3_output_path))

Configuration

In [ ]:
# create unique job name 
job_name_prefix = 'rl-hvac-chainer'

# TODO: eplus env cannot work on local.
local_mode = False

instance_type = "local" if local_mode else "ml.m4.4xlarge"

try:
    role = sagemaker.get_execution_role()
except:
    role = get_execution_role()

print("Using IAM role arn: {}".format(role))

# Only run from SageMaker notebook instance
if local_mode:
    !/bin/bash ./common/setup.sh

hvac用のdockerコンテナをビルドします。 / build docker container for hvac sample.

* **同じディレクトリにあるDockerfileの先頭を自分でビルドした sagemaker-rl-chainerrl に書き換えて下さい。**
/ **Please replace to your ECR repository on top of Dockerfile.**
* sagemaker-rl-chainerrlのビルド方法は[こちら](https://gist.github.com/ikeyasu/71a4b07ce6ecc7465c4e984ac5f5c855) / Please refer to [here](https://gist.github.com/ikeyasu/71a4b07ce6ecc7465c4e984ac5f5c855) for building sagemaker-rl-container.

例(Example)：
```
ARG CPU_OR_GPU
ARG AWS_REGION
FROM xxxx.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-rl-chainer:chainerrl0.5.0-${CPU_OR_GPU}-py3
```

`xxxx.dkr.ecr` で始まる部分を置き換えます。 / Please replace the start with `xxx.dkr.ecr`.

上記が終わったら以下を実行し、hvac 用のDockerイメージをビルドして下さい

In [ ]:
cpu_or_gpu = 'gpu' if instance_type.startswith('ml.p') else 'cpu'
repository_short_name = "sagemaker-hvac-chainerrl-%s" % cpu_or_gpu
docker_build_args = {
    'CPU_OR_GPU': cpu_or_gpu, 
    'AWS_REGION': boto3.Session().region_name,
}
custom_image_name = build_and_push_docker_image(repository_short_name, build_args=docker_build_args, dockerfile='Dockerfile')
print("Using ECR image %s" % custom_image_name)

上記が終わったら、学習を開始します。

In [ ]:
from sagemaker.chainer.estimator import Chainer

print("Local mode." if local_mode else "Launching training instance.")

# enegypower cannot work with ChainerRL if global step count is not multiple of episode len.
# 288 is 12 * 24
EPISODE_LEN = 288
chainer_estimator = Chainer(entry_point='train_chainerrl_dqn.py',
                            source_dir="src",
                            dependencies=["common/sagemaker_rl"],
                            role=role,
                            sagemaker_session=None if local_mode else sage_session,
                            image_name=custom_image_name,
                            framework_version="5.0.0",
                            train_instance_count=1,
                            train_instance_type=instance_type,
                            output_path=s3_output_path,
                            base_job_name=job_name_prefix,
                            hyperparameters={
                                'gpu': -1,
                                'steps': EPISODE_LEN * 3,
                                'eval-interval': EPISODE_LEN * 2,
                                'eval-n-runs': 1,
                            })

chainer_estimator.fit(wait=False)